In [18]:
import pandas
import pandas
import re
from spacy.lang.en import English
from spacy.vocab import Vocab

## Identifying OOV

In [107]:
class oovChecker():
    
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"]) # using default tokeniser with NER
        with open('./uncased_L-4_H-512_A-8/vocab.txt') as f:
            # if want to remove '[unusedXX]' from vocab
            # words = [line.rstrip() for line in f if not line.startswith('[unused')]
            words = [line.rstrip() for line in f]
        self.vocab = Vocab(strings=words)
 
        
    ## query --> "aa bb cc..."
    def misspellIdentify(self, query):
        """
        At present, All the following criteria should be met for word to be misspelled
        1. Should not in our vocab
        2. should not be a Person
        3. Should not be a number
        """
        doc = self.nlp(query)
        misspell = []
        for token in doc:
            if((token.text.lower() not in vocab) and 
               (token.ent_type_ != 'PERSON') and 
               (not token.like_num)):
                
                misspell.append(token)
                
        return misspell
    
    def candidateGenerator(self, misspellings):
        return {"misspell-1":["candidate-1","candidate-2"]}
    
    def candidateRanking(self, misspellingsDict):
        return {"misspell-1":"Best_candidate"}
        
        

In [109]:
checker = oovChecker()
checker.misspellIdentify("this is my India. My name is Rajat Goel. I use tru. Net income was $9.4 million compared to the prior year of $2.7 milion. my number is 5362722.2")


[tru, milion]

In [104]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"])
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)

doc = nlp("this is my pakistan. My name is Rajat Goel. I use tru. Net income was $9.4 million compared to the prior year of $2.7 milion. my number is 5362722.2")

        
from spacy.vocab import Vocab
with open('./uncased_L-4_H-512_A-8/vocab.txt') as f:
    # if want to remove '[unusedXX]' from vocab
    # words = [line.rstrip() for line in f if not line.startswith('[unused')]
    words = [line.rstrip() for line in f]
    
vocab = Vocab(strings=words)
# for i in tokens:
#     print(i.text.lower() in vocab, i.text)


# a= [token for token in doc if (token.text.lower() not in vocab  & token.ent_type != 'PERSON') ]

lst = []
for token in doc:
    print(token.text, token.ent_type_)
    if((token.text.lower() not in vocab) and (token.ent_type_ != 'PERSON') and (not token.like_num)):
#         print(token.text, token.ent_type_, token.is_digit, token.like_num)
        lst.append(token)
        
print(lst)
# print(a[0].i,a[0], a[0].ent_type)

# print([(ent.text, ent.label_) for ent in doc.ents])

print(len(vocab))
"apple" in vocab

this 
is 
my 
pakistan 
. 
My 
name 
is 
Rajat PERSON
Goel PERSON
. 
I 
use 
tru 
. 
Net 
income 
was 
$ MONEY
9.4 MONEY
million MONEY
compared 
to 
the DATE
prior DATE
year DATE
of 
$ 
2.7 MONEY
milion 
. 
my 
number 
is 
5362722.2 CARDINAL
[tru, milion]
30522


True

In [100]:
len(nlp.vocab)

57852